In [1]:
initial_network_file = 'data/filtered_network.gpkg'
output_file = 'data/output.gpkg'
epsg = 32618

In [2]:
import geopandas

initial_edges = geopandas.read_file(initial_network_file, layer="edges")
initial_nodes = geopandas.read_file(initial_network_file, layer="nodes")

/home/liberostelios/Development/tudelft3d/roads_generator/venv/lib/python3.9/site-packages/geopandas/geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [3]:
import osmnx as ox

initial_edges = initial_edges.set_index(['u', 'v', 'key'])
initial_network = ox.graph_from_gdfs(initial_nodes, initial_edges)

In [22]:
import networkx as nx

nx.number_weakly_connected_components(initial_network)

104

In [24]:
len(initial_edges)

66623

In [23]:
roadsAreal = ""
OSM = ""
carriageways = ""

# Area of convex hull
# Number of polygons
# Number of LoD1 edges
# Directionality of LoD1 edges (with graph)
# Polygon to LoD1 edge ratio
# Polygon area mean + std + median
# LoD1 edge length mean + std + median
# LoD1 edge total length
# LoD1 edge density (total edge length/area)
# Area to LoD1 length ratio
# Percentage overlap between linear and areal

# Road width mean + std + median
# Number of dual carriageways + proportion of all roads
# Number of LoD2 edges + percentage increase from OSM
# LoD2 edges length mean + std + median
# LoD2 edge total length
# LoD2 edge density (total edge length/area)
# Change in length between LoD1 and LoD2

# Number of intersections 
# Intersection Density
# Number of edges per node average + std + median + mode
# Number of 4 dual carriageways
# Number of 3 duals + 1 single
# Number of 3 duals
# Number of 1 dual + 2 singles
# Number of 2 duals + 1 single
# Number of dead ends LoD1 + proportion of all nodes
# Number of dead ends LoD2 + percentage increase + proportion of all nodes

# LoD1 connected components
# Percentage of edges in largest CC
# LoD2 CC
# Percent of edges in largest CC